#### **Project Details**

**Guided Project**

**Due Date:**  23/04/2025

**Contributors:** 
* Kyle Gregory Smith (577999)
* Caitlin Burnett (600330)
* Paul-Dieter Brandt (600948)
* Christoffel Martinus (577963)

## Data-Driven Student Support at BrightPath Academy (TBD)

### 1. Problem Statement
BrightPath Academy is committed to providing a well-rounded education that nurtures both academic excellence and personal growth. However, the school faces several challenges that hinder its ability to fully support every student. Currently, **the absence of real-time insights** means that struggling students may go unnoticed until its too late, if no form of intervention is administered to help them. Additionally, educators **lack targeted support strategies**, making it difficult to personalize academic assistance such as tutoring or mentoring.

While extracurricular activities are encouraged, **there is limited data** on their impact on academic performance, preventing educators from leveraging these programs to enhance student success. Moreover, the school collects vast amounts of student data, but **without a centralized, intuitive platform**, translating this information into actionable insights for student support remains a challenge.

To address these issues, BrightPath Academy is working to implement a **data-driven system** that identifies at-risk students in real time, enables educators to personalize interventions based on student needs, and provides clear insights into the impact of extracurricular involvement. By doing so, the academy **aims to enhance student outcomes, streamline academic support, and foster a more effective learning environment** for all learners.

### 2. Hypothesis


### 3. Importing Libraries & Data

In [ ]:
#Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import warnings

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

#Import csv files
df_original = pd.read_csv('../data/Student_performance_data.csv')               #.. goes up in the directory
df_original_copy = df_original.copy()                                           #Use throughout 

### 4. Understanding the data

#### Data Attributes:

#### Observations/Insights


### Dataset Data Types:

#### Observations/Insights

### Dataset Shape:

#### Observations/Insights

### 5. Exploring the Data

#### A. Univariate Analysis:
- Brief Overview of Univariate Analysis

#### B. Bi-variate Analysis:
- Brief Overview of Bi-variate Analysis

### 6. Missing Values and Outliers

### 7. Feature Engineering

### 8. First Model Development

#### Evaluate First Model Metrics

### 9. Second Model Development

#### Evaluate Second Model Metrics

### 10. Comparison of Models

### 11. Web Application